# Filter data based on keywords

## Introduction
The aim of this notebook is to read a set of keywords and a set of scraped data and filter out all non-Covid-19 related entries/rows.

## Import libraries and set up defaults

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#%xmode Verbose
# Set global default figure size
plt.rc('figure', figsize=(20, 12)) # It's nice with figures that fill the whole space in width
# Show maximum of 8 rows when printing dataframes
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 8
# Show only 4 digits when printing floating point number
np.set_printoptions(precision=4, suppress=True)

## Two sets of keywords

### Key words from [Zhang Lab](https://zhanglab.ccmb.med.umich.edu/COVID-19/) 

#### Read in the keywords

In [2]:
path = "/data/01_raw/Covid-19_keywords/"
key_words_df = pd.read_csv(path + "SARS-COV-2_Protein_Names.tsv",
                           sep = '\t',
                           header = 0,
                           usecols = ['ORF','Gene','Gene2','Full_Name'] # Dropping: 'Function', Source and a url
                          )
key_words_df

,ORF,Gene,Gene2,Full_Name
0,ORF1AB,nsp1,NaN,Host translation inhibitor nsp1
1,ORF1AB,nsp2,NaN,Non-structural protein 2
2,ORF1AB,nps3,NaN,Papain-like proteinase
3,ORF1AB,nps4,NaN,Non-structural protein 4
...,...,...,...,...
20,ORF7A,NaN,NaN,Protein 7a
21,ORF8,NaN,NaN,Protein 8
22,N,NaN,NaN,Nucleoprotein
23,ORF10,NaN,NaN,3` UTR


#### Create a unique Python list of keywords

In [3]:
first_term = (key_words_df['ORF']
              .dropna() # Drop np.nan:s
              .unique() # Filter all non-unique values
              .tolist() # Make a python list
             ) + \
key_words_df['Gene'].dropna().unique().tolist() + \
key_words_df['Gene2'].dropna().unique().tolist() + \
key_words_df['Full_Name'].dropna().unique().tolist()
print(first_term)

['ORF1AB', 'S', 'ORF3A', 'E', 'M', 'ORF6', 'ORF7A', 'ORF8', 'N', 'ORF10', 'nsp1', 'nsp2', 'nps3', 'nps4', 'nsp5', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', 'RDRP', 'Hel', 'Exon', 'NendoU', "2'-O-MT", 'Spike', '3CL-PRO', 'Spike trimeric complex (S1, S2, S`)', 'Host translation inhibitor nsp1', 'Non-structural protein 2', 'Papain-like proteinase', 'Non-structural protein 4', '3C-like proteinase', 'Non-structural protein 6', 'Non-structural protein 7', 'Non-structural protein 8', 'Non-structural protein 9', 'Non-structural protein 10', 'RNA-Directed RNA Polymerase', 'Helicase', 'Proofreading exoribonuclease (Guanine-N7 methyltransferase)', 'Uridylate-specific endoribonuclease', "2'-O-methyltransferase", 'Spike surface glycoprotein (monomer)', 'Protein 3a', 'Envelope small membrane proteins', 'Membrane protein', 'Protein 6', 'Protein 7a', 'Protein 8', 'Nucleoprotein', '3` UTR']


### Key words for filtering scraped data from [Mendeley Database](https://data.mendeley.com/)

In [30]:
first_term = ["sars-cov-2", 
              "sars-cov2", 
              "covid19", 
              "covid-19", 
              "sars", 
              "coronavirus", 
              "ncov2019", 
              "ncov-2019",
              "nCov2019",
              "nCov-2019", 
              "COVID2019"]
second_term = ["molecular dynamics", 
               "trajectories", 
               "dynamics simulation", 
               "trajectory", 
               "molecular docking"]

## Read in Scraped data

In [17]:
sc_path = "data/01_raw/Scraped/"
sc_data = pd.read_csv(sc_path + "dcd_trayectories_figshare_data.csv",
                      sep = ",",
                      header = 0,
                      index_col = 0
                     )
sc_data

,Title,Author,Categories,Keywords,Description
0,GTT-1-protein-005.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
1,GTT-1-protein-012.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
2,GTT-1-protein-013.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
3,GTT-1-protein-019.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
...,...,...,...,...,...
36,GTT-1-protein-065.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
37,GTT-1-protein-069.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
38,GTT-1-protein-072.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
39,GTT-1-protein-078.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain


### Check that the data

In [18]:
sc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        40 non-null     object
 1   Author       40 non-null     object
 2   Categories   40 non-null     object
 3   Keywords     40 non-null     object
 4   Description  40 non-null     object
dtypes: object(5)
memory usage: 1.9+ KB


In [19]:
sc_data.describe()

,Title,Author,Categories,Keywords,Description
count,40,40,40,40,40
unique,40,2,1,3,3
top,GTT-1-protein-001.dcd,Kresten Lindorff-Larsen,['Structural Biology (incl. Macromolecular Mod...,['molecular dynamics'],Trajectories dataTrajectories of WW domain
freq,1,39,40,38,38


## Initialise a boolen pd.Series
The purpose of pd.Series name `found` is to hold a boolen index rows with matches (if a certain keyword is found in the particular entry).

In [32]:
falses = np.zeros(len(sc_data["Title"]), dtype=bool) # https://stackoverflow.com/a/21174962
found1 = pd.Series(data = falses,
                   dtype = bool)
found2 = pd.Series(data = falses,
                   dtype = bool)

## Find all indexes with a match

### Find matches in the first column

In [33]:
first_column_to_search_in = "Keywords"
for word in first_term:
    # Find out if the current search term can be found in the column
    cur_match = sc_data[first_column_to_search_in].str.contains(word) # https://stackoverflow.com/a/15333283
    # Join the found matches to one Series
    found1 = found1 | cur_match

In [34]:
found1.value_counts()

False    39
True      1
dtype: int64

### Find matches in the second column

In [25]:
second_column_to_search_in = "Description"
for word in second_term:
    # Find out if the current search term can be found in the column
    cur_match = sc_data[second_column_to_search_in].str.contains(word) # https://stackoverflow.com/a/15333283
    # Join the found matches to one Series
    found2 = found2 | cur_match

In [11]:
found2

0    False
1    False
2     True
3     True
dtype: bool

### Join the matches from both columns

In [12]:
joined = found1 & found2
joined

0    False
1    False
2    False
3     True
dtype: bool

## Select only rows with matches in both columns

### When searching only with the first term

In [35]:
sc_data[found1]

,Title,Author,Categories,Keywords,Description
21,1 microsecond simulation of nCoV protease 6LU7...,Lenin Dominguez-Ramirez,['Structural Biology (incl. Macromolecular Mod...,"['nCov2019, nCov-2019, COVID2019, Drug Repurpo...",This protein was simulated without its cocryst...


### When searching with two terms

In [13]:
sc_data[joined]

,col_1,col_2
3,sars-cov-2,molecular dynamics
